# Loop in given directory to find pdf file according to regexp and merge all pdf file in complete.pdf file 

In [36]:
# importing required modules
from tabula import read_pdf
import pandas as pd
import numpy as np 
from tabulate import tabulate
import numpy as np
import os
import glob 
from dateutil.parser import parse
import PyPDF2
from PyPDF2 import PdfReader , PdfWriter, PdfMerger
from datetime import timedelta
import datetime as dtt


os.chdir(r'C:\Users\Halim\OneDrive\02 - Professional Data\02 - Technique\02 - Python\03 - Projets\ProfessionnalAccountig\00-Input')
current_directory = os.getcwd()

ListExtraitsCompte = [f for f in glob.glob("*EXTRAIT_COMPTE*.pdf")]

if not ListExtraitsCompte: 
        print ("No file founded") 
else: 
        merger = PdfMerger() 
        for file in ListExtraitsCompte: 
            merger.append(file) 
        merger.write("Complete.pdf") 
        merger.close() 
                   







# Populate dataframe by complete.pdf file generated above

In [37]:

df = pd.concat(read_pdf('Complete.pdf',pages='all',multiple_tables=True,encoding="latin_1" ))
df.size

print (df) 


          Date Date valeur                      Opération Débit EUROS  \
0          NaN         NaN  SOLDE CREDITEUR AU 31/12/2023         NaN   
1   01/01/2024  01/01/2024         VIR INST RENUM 2023/12    3.000,00   
2          NaN         NaN               VH40013A4GLYQ501         NaN   
3   01/01/2024  01/01/2024     VIR INST LOCAL PRO 2023/12      500,00   
4          NaN         NaN               VH40013ACY2PZQ01         NaN   
..         ...         ...                            ...         ...   
7          NaN         NaN               VH4031415V29A101         NaN   
8   31/01/2024  31/01/2024   VIR INST INDEMN KILO 2024/01    1.347,00   
9          NaN         NaN               VH403141MUAST201         NaN   
10         NaN         NaN                            NaN   15.435,18   
11   Réf : 001         NaN  SOLDE CREDITEUR AU 31/01/2024         NaN   

   Crédit EUROS            Unnamed: 0  
0     42.852,38                   NaN  
1           NaN                   NaN  
2  

c:\Users\Halim\AppData\Local\Programs\Python\Python312\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [38]:
print (type(df) )
#df.to_excel(r'E:\ADMINISTRATIF\PRO\\3A CONSULTING\BILAN 2024\BANQUE\Output.xlsx')


<class 'pandas.core.frame.DataFrame'>


In [39]:
# get dataframe columns 
df.columns


Index(['Date', 'Date valeur', 'Opération', 'Débit EUROS', 'Crédit EUROS',
       'Unnamed: 0'],
      dtype='object')

In [41]:
# drop last columns 
# Axis = 1 in data frame means columns 
df.drop(df.columns[[5]],axis=1,inplace=True)
df.columns

Index(['Date', 'Date valeur', 'Opération', 'Débit EUROS', 'Crédit EUROS'], dtype='object')

In [42]:
# get statitstics of dataframe 
#df.describe(include='all')

# get first 10 dataframe's rows 
#df.head(n=10)

# get last 10 dataframe's rows 
#df.tail(n=10)


# columns types 
df.dtypes 
# get numbers of empty rows for each dataframe's columns 
df.isnull().sum()


Date            62
Date valeur     63
Opération        1
Débit EUROS     63
Crédit EUROS    88
dtype: int64

In [43]:
# Delete all rows in dataframe when opération column contains SOLDE or Total 
df = df[df["Opération"].str.contains("SOLDE")==False]  
df = df[df["Opération"].str.contains("Total")==False]  

# convert data columns 
df[["Date", "Date valeur"]] = df[["Date", "Date valeur"]].apply(lambda x: pd.to_datetime(x, errors = 'coerce', format='%d/%m/%Y')) 
df 

# reset daztaframe index 
df.reset_index(drop=True, inplace=True)
df

,Date,Date valeur,Opération,Débit EUROS,Crédit EUROS
0,2024-01-01,2024-01-01,VIR INST RENUM 2023/12,"3.000,00",NaN
1,NaT,NaT,VH40013A4GLYQ501,NaN,NaN
2,2024-01-01,2024-01-01,VIR INST LOCAL PRO 2023/12,"500,00",NaN
3,NaT,NaT,VH40013ACY2PZQ01,NaN,NaN
4,2024-01-02,2024-01-01,INTERETS/FRAIS,"19,84",NaN
...,...,...,...,...,...
84,2024-01-31,2024-01-31,VIR RENUM 2024/01,"3.500,00",NaN
85,2024-01-31,2024-01-31,VIR INST FRAIS LOCAL PRO 2024/0,"500,00",NaN
86,NaT,NaT,VH4031415V29A101,NaN,NaN
87,2024-01-31,2024-01-31,VIR INST INDEMN KILO 2024/01,"1.347,00",NaN


In [44]:
# formatage colonnes 
df[["Débit EUROS", "Crédit EUROS"]] = df[["Débit EUROS", "Crédit EUROS"]].apply(lambda x: x.str.replace('.',''))
df[["Débit EUROS", "Crédit EUROS"]] = df[["Débit EUROS", "Crédit EUROS"]].apply(lambda x: x.str.replace(',','.'))
df[["Débit EUROS", "Crédit EUROS"]] = df[["Débit EUROS", "Crédit EUROS"]].apply(lambda x:pd.to_numeric(x))
df.dtypes


df['Débit EUROS'] = np.where(pd.isnull(df['Débit EUROS']) &  ~pd.isnull(df['Crédit EUROS']), 0.0, df['Débit EUROS'])
df['Crédit EUROS'] = np.where(pd.isnull(df['Crédit EUROS']) &  ~pd.isnull(df['Débit EUROS']) , 0.0, df['Crédit EUROS'])
df

,Date,Date valeur,Opération,Débit EUROS,Crédit EUROS
0,2024-01-01,2024-01-01,VIR INST RENUM 2023/12,3000.00,0.0
1,NaT,NaT,VH40013A4GLYQ501,NaN,NaN
2,2024-01-01,2024-01-01,VIR INST LOCAL PRO 2023/12,500.00,0.0
3,NaT,NaT,VH40013ACY2PZQ01,NaN,NaN
4,2024-01-02,2024-01-01,INTERETS/FRAIS,19.84,0.0
...,...,...,...,...,...
84,2024-01-31,2024-01-31,VIR RENUM 2024/01,3500.00,0.0
85,2024-01-31,2024-01-31,VIR INST FRAIS LOCAL PRO 2024/0,500.00,0.0
86,NaT,NaT,VH4031415V29A101,NaN,NaN
87,2024-01-31,2024-01-31,VIR INST INDEMN KILO 2024/01,1347.00,0.0


In [45]:
# chercher les doublons 
doublons = df[df.duplicated()]
doublons

for index in doublons.index:
    print (index)  
    if pd.notnull(df.iloc[index, 1] ):
          df.iloc[index, 1] = df.iloc[index, 1]  +  timedelta(days=1)
    


48
58
77
80
83


In [47]:
#si deux lignes se suivent 
# colonne 1
# 02/02/2024
# NaT

# deviens : 
# colonne 1 
#02/02/2024
#02/02/2024

df.ffill(inplace=True) 
df

#Group by Date , Date Valeur , Débit Euros, Crédit Euros et concaténer la colonne #Opération 
df3 = df.groupby (['Date', 'Date valeur', 'Débit EUROS', 'Crédit EUROS'])['Opération'].apply (' '.join).reset_index() 
df3

df3.size 



145

In [49]:
# add a new column 

# Add a new column 'B' using a switch-case-like statement
df3["Catégorie"] = np.where(df3["Opération"].str.contains("FRAIS LOCAL", case=False), "FRAIS LOCAL PRO", 
                                np.where(df3["Opération"].str.contains("INDEMN",  case=False), "INDEMNITE KILOMETRIQUE",
                                         np.where(df3["Opération"].str.contains("A2 CONSULTING",  case=False), "REGLEMENT PRESTATION",
                                  np.where(df3['Opération'].str.contains("06XXXXX897",  case=False), "TELEPHONIE MOBILE",
                                     np.where(df3['Opération'].str.contains("AGIPI",  case=False), "ASS PREVOYANCE -- AGIPI",
                                              np.where(df3['Opération'].str.contains(" DGFIP IS",  case=False), "IMPOTS - IS",
                                              np.where(df3['Opération'].str.contains("RENUM",  case=False), "RENUMERATION",
                                            np.where(df3['Opération'].str.contains("FREE MOBILE",  case=False), "TELEPHONIE MOBILE - FREE",
                                                     np.where(df3['Opération'].str.contains('DGFIP TVA',  case=False), 'IMPOTS -- TVA',  
                                                            np.where(df3['Opération'].str.contains('09XXXXX206',  case=False), 'TELEPHONIE FIXE',       
                                                                np.where(df3['Opération'].str.contains('DIRECT ASSURANCE',  case=False), 'ASSURANCE - DIRECT ASSURANCE',       
                                                                        np.where(df3['Opération'].str.contains('INTERETS',  case=False), 'BANQUE - FRAIS BANCAIRE',    
                                                                            np.where(df3['Opération'].str.contains('AMAZON',  case=False),   'AMAZON - ACHATS',  
np.where(df3['Opération'].str.contains('MMA IARD',  case=False), 'ASS PREVOYANCE -- MMA',  
                                                              np.where(df3['Opération'].str.contains('AR CONSEIL,  case=False'), 'FRAIS COMPTABLE',
                                        np.where(df3['Opération'].str.contains('URSSAF',  case=False), 'URSSAF',    
                                         np.where(df3['Opération'].str.contains('PAIEMENT CB',  case=False) & df3["Débit EUROS"]<50, 'RESTAURATION', 
np.where(df3['Opération'].str.contains('PAIEMENT PSC',  case=False), 'RESTAURTION',                      

                                                  
                                                                        np.where(df3['Opération'].str.contains('DIVIDENDES',  case=False), 'DIVIDENDES',"DIVERS")))))))))))))) )))) ) 

                     
                            
            
         

df3



,Date,Date valeur,Débit EUROS,Crédit EUROS,Opération,Catégorie
0,2024-01-01,2024-01-01,500.00,0.0,VIR INST LOCAL PRO 2023/12 VH40013ACY2PZQ01,RESTAURATION
1,2024-01-01,2024-01-01,3000.00,0.0,VIR INST RENUM 2023/12 VH40013A4GLYQ501,RENUMERATION
2,2024-01-02,2024-01-01,19.84,0.0,INTERETS/FRAIS,BANQUE - FRAIS BANCAIRE
3,2024-01-04,2024-01-04,106.93,0.0,PRLV SEPA AVANSSUR AVANSSUR DIRECT ASSURANCE 1...,ASSURANCE - DIRECT ASSURANCE
4,2024-01-08,2024-01-08,75.94,0.0,PRLV SEPA BOUYGUES TELECOM BOUYGUES TELECOM 06...,TELEPHONIE MOBILE
5,2024-01-09,2024-01-09,0.00,12600.0,VIR A2 CONSULTING A2 CONSULTING 202311 / 2023-...,REGLEMENT PRESTATION
6,2024-01-09,2024-01-09,29.50,0.0,PAIEMENT PSC 0801 75 PARIS 13 LA PIOLACARTE 1516,RESTAURATION
7,2024-01-09,2024-01-09,43.52,0.0,PAIEMENT CB 0601 AMSTERDAM UBER TRIPCARTE 1516,RESTAURATION
8,2024-01-10,2024-01-01,41.25,0.0,"FACT SGT24061550000225 DONT TVA 0,58EUR",RESTAURATION
9,2024-01-10,2024-01-10,21.10,0.0,PAIEMENT PSC 0901 PARIS LA ROSERAIE /KDR CARTE...,RESTAURATION


In [50]:
df3

,Date,Date valeur,Débit EUROS,Crédit EUROS,Opération,Catégorie
0,2024-01-01,2024-01-01,500.00,0.0,VIR INST LOCAL PRO 2023/12 VH40013ACY2PZQ01,RESTAURATION
1,2024-01-01,2024-01-01,3000.00,0.0,VIR INST RENUM 2023/12 VH40013A4GLYQ501,RENUMERATION
2,2024-01-02,2024-01-01,19.84,0.0,INTERETS/FRAIS,BANQUE - FRAIS BANCAIRE
3,2024-01-04,2024-01-04,106.93,0.0,PRLV SEPA AVANSSUR AVANSSUR DIRECT ASSURANCE 1...,ASSURANCE - DIRECT ASSURANCE
4,2024-01-08,2024-01-08,75.94,0.0,PRLV SEPA BOUYGUES TELECOM BOUYGUES TELECOM 06...,TELEPHONIE MOBILE
5,2024-01-09,2024-01-09,0.00,12600.0,VIR A2 CONSULTING A2 CONSULTING 202311 / 2023-...,REGLEMENT PRESTATION
6,2024-01-09,2024-01-09,29.50,0.0,PAIEMENT PSC 0801 75 PARIS 13 LA PIOLACARTE 1516,RESTAURATION
7,2024-01-09,2024-01-09,43.52,0.0,PAIEMENT CB 0601 AMSTERDAM UBER TRIPCARTE 1516,RESTAURATION
8,2024-01-10,2024-01-01,41.25,0.0,"FACT SGT24061550000225 DONT TVA 0,58EUR",RESTAURATION
9,2024-01-10,2024-01-10,21.10,0.0,PAIEMENT PSC 0901 PARIS LA ROSERAIE /KDR CARTE...,RESTAURATION


In [51]:
df3.to_csv(r"C:\Users\Halim\OneDrive\02 - Professional Data\02 - Technique\02 - Python\03 - Projets\ProfessionnalAccountig\00-Input\Comptes_2023.csv", sep=";", encoding="Latin_1") 